In [ ]:
import pandas as pd
import polars as pl
import numpy  as np
import networkx as nx
import random
from rtsvg import *
rt = RACETrack()
edges_filename  = '../../data/stanford/facebook/348.edges'
_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
my_df  = pl.DataFrame(_lu_)
g      = rt.createNetworkXGraph(my_df, [('fm','to')])
my_pos = nx.spring_layout(g)
#pos = rt.hyperTreeLayout(g)
#pos = nx.kamada_kawai_layout(g)
params = {'df':my_df, 'relationships':[('fm','to')], 'pos':my_pos, 'w':512, 'h':512}

In [ ]:
def chordRoutedGraph(df, relationships, pos, k=6, iters=100, w=384, h=384, bounds_percent=0.2, draw_neighborhoods=True):
    # Create a link for transformation information
    _link_ = rt.link(df, relationships=relationships, pos=pos, w=w, h=h, bounds_percent=bounds_percent)
    _link_.renderSVG() # force the rendering and related transformations to occur

    # Flatten the nodes out -- assumes that all nodes (and only nodes) are in the pos dictionary
    _node_ls_  = list(pos.keys())
    _node_to_i_ = {}
    for i in range(len(_node_ls_)): _node_to_i_[_node_ls_[i]] = i
    _node_sxy_ = []
    for _node_ in _node_ls_: 
        sx, sy = _link_.xT(pos[_node_][0]), _link_.yT(pos[_node_][1])
        _node_sxy_.append((sx, sy))
    sx_min, sx_max, sy_min, sy_max = sx, sx, sy, sy
    for _node_ in _node_ls_:
        sx_min, sx_max = min(sx_min, sx), max(sx_max, sx)
        sy_min, sy_max = min(sy_min, sy), max(sy_max, sy)

    # Make random cluster centers
    cluster_centers = {}
    for i in range(k): 
        sx, sy = random.random() * (sx_max - sx_min) + sx_min, random.random() * (sy_max - sy_min) + sy_min
        cluster_centers[i] = (sx, sy)

    # Iterate K-Means
    for _iter_ in range(iters):
        # Assign nodes to their closest center
        center_assignments = {}
        for j in range(len(_node_ls_)):
            _node_ = _node_ls_[j]
            min_dist       = (_node_sxy_[j][0] - cluster_centers[0][0])**2 + (_node_sxy_[j][1] - cluster_centers[0][1])**2
            closest_center = 0
            for i in range(1, k):
                dist = (_node_sxy_[j][0] - cluster_centers[i][0])**2 + (_node_sxy_[j][1] - cluster_centers[i][1])**2
                if dist < min_dist:
                    min_dist       = dist
                    closest_center = i
            if closest_center not in center_assignments: center_assignments[closest_center] = []
            center_assignments[closest_center].append(_node_)
        # If there are any centers without nodes, assign a random node to them
        for i in range(k): 
            if i not in center_assignments: center_assignments[i] = [random.choice(_node_ls_)]
        # Update centers
        for i in range(k): 
            sx, sy = 0, 0
            for _node_ in center_assignments[i]: 
                sx, sy = sx + _node_sxy_[_node_to_i_[_node_]][0], sy + _node_sxy_[_node_to_i_[_node_]][1]
            cluster_centers[i] = (sx/len(center_assignments[i]), sy/len(center_assignments[i]))

    # Track nodes to their cluster centers
    node_to_cluster_center = {}
    for i in center_assignments:
        for _node_ in center_assignments[i]: 
            node_to_cluster_center[_node_] = i

    # Create the Voronoi diagram from the cluster centers
    voronoi_pts = []
    for i in cluster_centers: voronoi_pts.append(cluster_centers[i])
    nborhoods = rt.isedgarVoronoi(voronoi_pts, Box=[(0,_link_.h),(_link_.w,_link_.h),(_link_.w,0),(0,0)])

    # Create SVG
    distinguishable_colors = rt.co_mgr.brewerColors('qualitative', k)
    svg = [f'<svg x="0" y="0" width="{w}" height="{h}">']

    if draw_neighborhoods:
        for _poly_i_ in range(len(nborhoods)):
            _poly_ = nborhoods[_poly_i_]
            d      = f'M {_poly_[0][0]} {_poly_[0][1]}  '
            for i in range(1, len(_poly_)): d += f'L {_poly_[i][0]} {_poly_[i][1]} '
            d += 'Z'
            svg.append(f'<path d=\"{d}\" fill=\"{distinguishable_colors[_poly_i_]}\" />')

    _fm_, _to_ = relationships[0][0], relationships[0][1]
    for nborhood_i in center_assignments:
        in_nborhood = set(center_assignments[nborhood_i])
        _df_     = df.filter((pl.col(_fm_).is_in(in_nborhood)) | (pl.col(_to_).is_in(in_nborhood)))
        _df_map_ = {}
        for _node_ in node_to_cluster_center:
            if _node_ in in_nborhood: _df_map_[_node_] = _node_
            else:                     _df_map_[_node_] = -node_to_cluster_center[_node_]
        _df_ = _df_.with_columns(pl.col(_fm_).replace_strict(_df_map_).alias('__fm__'), 
                                 pl.col(_to_).replace_strict(_df_map_).alias('__to__'))
        _xy_   = cluster_centers[nborhood_i]
        _cd_r_ = 64
        _svg_ = rt.chordDiagram(_df_, [('__fm__','__to__')], x_view=_xy_[0] - _cd_r_, y_view=_xy_[1] - _cd_r_, w=2*_cd_r_, h=2*_cd_r_, draw_border=False)
        svg.append(_svg_._repr_svg_())

    svg.append('</svg>')
    return ''.join(svg)

rt.tile([chordRoutedGraph(k=10, draw_neighborhoods=False, **params), rt.link(**params)])

In [ ]:
help(rt.chordDiagram)